CMPT 2400: Exploratory Data Analysis Data Project
Prepared by Laura Brin, Sandra Alex & Annabell Rodriguez

loading libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier 
from sklearn.model_selection import train_test_split
import calendar
from calendar import month_name as mn

# This makes it so we are able to see 100 rows when displaying the data
pd.set_option("display.max_rows", 100)

loading datasets

In [ ]:
pass_df=pd.read_csv("dataset/International_Report_Passengers.csv")
depart_df=pd.read_csv("dataset/International_Report_Departures.csv")
world_airp_codes_df=pd.read_csv("dataset/world_airport_codes.csv")

### Posed Problem: Flight Delay Propagation Mitigation

International travel involves a web of interconnected airports in hundreds of countries every day of the year. 
The ripple effect caused by a cancelled or delayed flight can cause issues with missed connections, missing baggage, carrier fines, reimbursed customers and staffing issues.

According to the Federal Aviation Administration, Delay Propagation occurs when three conditions are met simultaneously (https://aspm.faa.gov/aspmhelp/index/Delay_Propagation.html#:~:text=Delay%20propagation%20occurs%20when%20a,identified%20by%20a%20tail%20number.):

- A flight arrives late at an airport.
- A flight departs late in subsequent stages.
- A flight arrives late at the next destination.


### Posed Solution

We would like to pitch a ML solution using this dataset that would assist with real time decisions for domestic flight delays. When domestic flight centers experience multiple delays, air traffic decision makers can use the model to help predict which flights should be prioritized for take-off to reduce flight delay propagation into international connecting flights. It will do this by looking at the relationship between the flight's intended landing airport, the number of international airports that site connects with, how many flights leave that site, the region of the airport and the time of year.

### Observing Departures Dataset

This dataset contains data on all the flights between US gateways and non-US gateways. It is a record of international flights departing US and can be used to highlight busiest airports, and peak times for flight volume

There are multiple abbreviations used in this section:
* DOT: Department of Transportation
* FAA: Federal Aviation Administration
* IATA: International Air Transportation Association
* ICAO: International Civil Aviation Organization

    #Laura

#### Features

Date- in MM/DD/YYYY format

Year

Month

> usg_apt_id: US Gateway Airport ID- assigned by US DOT to identify airport

> usg_apt: US Gateway Airport Code- usually assigned by IATA but in absence of IATA designation, may show FAA-assigned code. For full list of World Airport codes see the Bureau of Transportation Statistics: https://www.bts.gov/topics/airlines-and-airports/world-airport-codes 

        These two features are related. They represent the numerical location code (US) and three letter code for location identification, respectively. These should correlate 1:1 except where FAA coding was used in the absence of IATA coding. 


usg_wac: US Gateway World Area code- assigned by US DOT to represent a geographic territory. 
* 1-99 USA, 
* 100-199 Central America, 
* 200-299 Caribbean, Bahamas and Bermuda, 
* 300-399 South America, 
* 400-499 Europe, 
* 500-599 Africa, 
* 600-699 Middle East, 
* 700-800 Far East/Asia, 
* 801-899 Antarctica, Australasia and Oceania, 
* 900-999 Canada and Greenland 
codes groupings from https://en.wikipedia.org/wiki/World_Area_Codes



> fg_apt_id: Foreign Gateway Airport ID-assigned by US DOT to identify an airport

> fg_apt: Foreign Gateway Airport Code- usually assigned by IATA but in absence of IATA designation, may show FAA assigned code

> fg_wac: Foreign Gateway World Area Code- Assigned by US DOT to represent territory. For code groups see above in usg_wac comments

        These three features are related. They represent the five digit numerical location code (US), three letter code for location identification (International), and three digit numerical location code (international), respectively.

> airlineid: Airline ID assigned by US DOT to identify an air carrier

> carrier: IATA assigned air carrier code. If carrier has no IATA code, ICAO- or FAA assigned code may be used. These are mixed letter/number codes. For full list of air carrier codes see the Bureau of Transportation Statistics: https://www.bts.gov/topics/airlines-and-airports/airline-codes 

        These two features are related. They represent the five digit numerical airline ID (US) and the two or three character air carrier id (international). These should correlate 1:1 except where IATA coding was absent. 

carriergroup: group code. 1=US domestic air carriers, 0=foreign air carriers

type: type of the metrics- this is a single code for this dataset= "Departures"

> Scheduled: metric flow by scheduled service operations. Scheduled flights are those commercially available for indivdual purchase

> Charter: metric flown by charter operations. Charter flights are booked by a group or consortium responsible for all seats on the flight. This is commonly reffered to as private flights

        These two features are related. Flights are listed as either scheduled or charter. Flights on the same day, from the same airline id, with the same take off and landing sites are recorded as a count metric

Total: scheduled+charter flight counts



        Notes: 
need to set date format correctly
year/month-numerical-any need for month to be in categorical?
need to check where more than 2 usg_apt is assigned to usg_apt_id and relabel
important pieces- US gatewayForeign Gateway and US_ world Area codeforeign world area code
year,month,
apt_ids and airlineid all as numeric-actually categorical
should discuss if we want to include charter flights as the problem statement is directed towards scheduled flights

In [ ]:
depart_df.head(20)

In [ ]:
depart_df.describe()

In [ ]:
depart_df.shape

In [ ]:
depart_df.dtypes

### Annabell

#### Correting data types

In [ ]:
# Change datatype of the date column
### departures
depart_df["data_dte"] = pd.to_datetime(depart_df["data_dte"])
### passengers
pass_df["data_dte"] = pd.to_datetime(pass_df["data_dte"])

# Change datatype of the categories columns
world_airp_code_type = pd.CategoricalDtype(categories=world_airp_codes_df['Code'].unique())
departure_type_type = pd.CategoricalDtype(categories=["Departures", "Passengers"])

### departures
depart_df['usg_apt'] = depart_df['usg_apt'].astype(world_airp_code_type)
depart_df['fg_apt'] = depart_df['fg_apt'].astype(world_airp_code_type)
depart_df['carrier'] = depart_df['carrier'].astype(world_airp_code_type)
depart_df['type'] = depart_df['carrier'].astype(departure_type_type)

### passengers
pass_df['usg_apt'] = pass_df['usg_apt'].astype(world_airp_code_type)
pass_df['fg_apt'] = pass_df['fg_apt'].astype(world_airp_code_type)
pass_df['carrier'] = pass_df['carrier'].astype(world_airp_code_type)
pass_df['type'] = pass_df['carrier'].astype(departure_type_type)


In [ ]:
### departures
depart_df.dtypes

In [ ]:
### passengers
pass_df.dtypes

Observing the correlation between the columns

In [ ]:
### departures
df_c = depart_df.corr(numeric_only=True)
df_c

In [ ]:
### passengers
pdf_c = pass_df.corr(numeric_only=True)
pdf_c

        We cannot appreciate an existing correlation between the numerical columns. The only association we can identify is between the scheduled column and the total column. Given that the total column displays the total of the scheduled and charter, this is to be expected. While it seems strange that there is no relationship between the charter and the sum. Let's look at the sum of these columns.

In [ ]:
### departures
print(depart_df['Charter'].sum())
print(depart_df['Scheduled'].sum())

In [ ]:
### passengers
print(pass_df['Charter'].sum())
print(pass_df['Scheduled'].sum())

        We can observe a significant disparity between them based on these totals. Since charter flights are less frequent, their occurrence has little impact on the total.

##### Analyzing seasonal patterns

Grouping the totals by date and analyzing them in time. The column data_dte represents a month because the day is always 01.

In [ ]:
### departures
plt.figure(figsize=(15,6))
out_total = depart_df.groupby('data_dte')['Total'].sum().reset_index(name ='Total')
sns.lineplot(x='data_dte',y='Total',data=out_total)
plt.title("Total vs Date")
plt.xlabel("Date")
plt.ylabel("Total")
plt.show()

In [ ]:
### departures
out_total.describe()

Analyzing Charter vs Scheduled

In [ ]:
### departures
plt.figure(figsize=(15,6))
out_charter = depart_df.groupby('data_dte')['Charter'].sum().reset_index(name ='Charter')
sns.lineplot(x='data_dte',y='Charter',data=out_charter)
plt.title("Charter vs Date")
plt.xlabel("Date")
plt.ylabel("Charter")
plt.show()


In [ ]:
### departures
out_charter.describe()

In [ ]:
### departures
plt.figure(figsize=(15,6))
out_scheduled = depart_df.groupby('data_dte')['Scheduled'].sum().reset_index(name ='Scheduled')
sns.lineplot(x='data_dte',y='Scheduled',data=out_scheduled)
plt.title("Scheduled vs Date")
plt.xlabel("Date")
plt.ylabel("Scheduled")
plt.show()

In [ ]:
### departures
out_scheduled.describe()

        We must evaluate charter departures separately since, as we have already seen, they are less frequent than scheduled departures. The graphics demonstrate how this situation causes the entire curve to be almost equivalent to the scheduled one.

        We are unable to identify a growth pattern in the chart of charter departures. Between 1990 and 1996, it grew; following that, it was relatively constant with some pikes until 2010, when it suddenly decreased. It then recovered, peaking in value in 2004, after which a steady drop started. This pattern can be traced back to 2010, when it suddenly increased at the beginning of 2020, reaching its greatest level since 1990, before beginning to drop in the months that followed. The maximum was 9382, and the average was about 5142. We can see some seasonality there; eventually, we'll look into it more thoroughly.

        On the graph of the scheduled departures, there is a clear seasonality and an upward trend. Between 2000 to 2004, there was a decline, which swiftly recovered. In 2020, we started to notice the start of the pandemic's effects. Sadly, there are just a few months of this year in the dataset.


##### Looking to the passengers dataframe

In [ ]:
### passengers
plt.figure(figsize=(15,6))
out_total = pass_df.groupby('data_dte')['Total'].sum().reset_index(name ='Total')
sns.lineplot(x='data_dte',y='Total',data=out_total)
plt.title("Total vs Date passenger dataset")
plt.xlabel("Date")
plt.ylabel("Total")
plt.show()

In [ ]:
### passengers
out_total.describe()

Analyzing Charter vs Scheduled

In [ ]:
### passengers
plt.figure(figsize=(15,6))
out_charter = pass_df.groupby('data_dte')['Charter'].sum().reset_index(name ='Charter')
sns.lineplot(x='data_dte',y='Charter',data=out_charter)
plt.title("Charter vs Date passenger dataset")
plt.xlabel("Date")
plt.ylabel("Charter")
plt.show()

In [ ]:
### passengers
out_charter.describe()

In [ ]:
### passengers
plt.figure(figsize=(15,6))
out_scheduled = pass_df.groupby('data_dte')['Scheduled'].sum().reset_index(name ='Scheduled')
sns.lineplot(x='data_dte',y='Scheduled',data=out_scheduled)
plt.title("Scheduled vs Date passenger dataset")
plt.xlabel("Date")
plt.ylabel("Scheduled")
plt.show()

In [ ]:
### passengers
out_scheduled.describe()

        The graphs show how comparable the patterns in the passenger dataset are to those in the departure dataset.

        In the case of charters, there is a propensity for this to decrease over time. With a little gain in the early years, roughly from 1990 to 1994, it then starts to decline. Some seasonality exists.

        However, the trend is upward for those that are scheduled. The reduction between 2000 and 2004 is also visible, but it subsequently resumed its upward trend until 2020, when it abruptly fell, as we now know due to the pandemic. There is a visible seasonality.

##### Analizing the last five years

In [ ]:
# month list
months = mn[1:]

### departures
max_year_depart = depart_df["Year"].max()
### passengers
max_year_pass = pass_df["Year"].max()

### departures
depart_df['month_category'] = depart_df['Month'].apply(lambda x: calendar.month_name[x])

### passengers
pass_df['month_category'] = pass_df['Month'].apply(lambda x: calendar.month_name[x])

# convert the column to categorical and ordered
### departures
depart_df["month_category"] = pd.Categorical(depart_df["month_category"], categories=months, ordered=True)
### passengers
pass_df["month_category"] = pd.Categorical(pass_df["month_category"], categories=months, ordered=True)

### departures
depart_df_last_5_years = depart_df.loc[(depart_df['Year'] >= (max_year_depart-4)) & (depart_df['Year'] <= max_year_depart)]
### passengers
pass_df_last_5_years = pass_df.loc[(pass_df['Year'] >= (max_year_pass-4)) & (pass_df['Year'] <= max_year_pass)]

In [ ]:
### departures
sns.relplot(kind='line', data=depart_df_last_5_years, x='month_category', y='Scheduled', hue='Year', aspect=2.5, marker='o', errorbar=None, height=5)
plt.title("Scheduled vs Month last five years (Departures dataset)")
plt.xlabel("Month")
plt.ylabel("Scheduled")
plt.show()

In [ ]:
### passengers
sns.relplot(kind='line', data=pass_df_last_5_years, x='month_category', y='Scheduled', hue='Year', aspect=2.5, marker='o', errorbar=None, height=5)
plt.title("Scheduled vs Month last five years (Passengers dataset)")
plt.xlabel("Month")
plt.ylabel("Scheduled")
plt.show()


        We can see the seasonality in these two graphs. In the case of the scheduled flights, there are several similarities between the two datasets. The largest numbers are in July and August, while the lowest values are in February. After beginning in January about the middle, it drops in February, climbs in March, and diminishes in April . Then, beginning in May, the amount rises through August before falling in September, grows a little in October, drops off in November, then picks back up in December. Over the past five years, there has been a recurring pattern.

        2020 is the exception. Due to the epidemic, it should have increased in March but instead fell to its lowest level.

In [ ]:
### departures
sns.relplot(kind='line', data=depart_df_last_5_years, x='month_category', y='Charter', hue='Year', aspect=2.5, marker='o', errorbar=None, height=5)
plt.title("Charter vs Month last five years (Departures dataset)")
plt.xlabel("Month")
plt.ylabel("Charter")
plt.show()

In [ ]:
### passengers
sns.relplot(kind='line', data=pass_df_last_5_years, x='month_category', y='Charter', hue='Year', aspect=2.5, marker='o', errorbar=None, height=5)
plt.title("Charter vs Month last five years (Passengers dataset)")
plt.xlabel("Month")
plt.ylabel("Charter")
plt.show()

        It differs between the two datasets for charter flights.

        With the exception of October 2018, which grew significantly more than the rest of the years, the departures dataset's seasonality can be appreciated during the first three years of analysis. However, 2019 follows a different pattern, and the values rise much more dramatically. Up to March, the final month we have, it seems to start recovering the old pattern in 2020. The values here are lowest in the initial years.

        In contrast, the passenger dataset's beginning years had the highest numbers, and 2019 shows a fall. 2020 is out of the question because of the enormous behavioural change brought on by the epidemic.



### Sandra